In [ ]:
import os
import openai

import fitz  # PyMuPDF

# load env
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# Initialize OpenAI API
openai.api_key = os.getenv("OPENAI_KEY")


In [ ]:
def extract_text_from_pdf(pdf_path):
    pdf_document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text += page.get_text()
    return text


In [ ]:
pdf_path = "your.pdf"
pdf_text = extract_text_from_pdf(pdf_path)

In [ ]:
if pdf_text == "":

    doc = fitz.open(pdf_path) 
    for page in doc: 
        pix = page.get_pixmap(matrix=fitz.Identity, dpi=None, 
                            colorspace=fitz.csRGB, clip=None, alpha=False, annots=True) 
        pix.save(f"{pdf_path[:-4]}-{page.number}.jpg")  # save file

In [ ]:
pdf_text

## Parsing PDF where the text is selectable

In [ ]:
prompt = f"""
You are an AI assistant that helps people understand the content of a PDF document.
You will be given a PDF document as raw text and you need to parse it carefully and return all the information in a JOSN format and nothing else.
"""

In [ ]:
gpt_messages = [
    {
        "role": "system",
        "content": prompt
    },
    {
        "role": "user",
        "content": pdf_text
    }
]

response = openai.chat.completions.create(
    model="gpt-4o",
    # response_format={ "type": "json_object" },
    messages=gpt_messages,
)



In [ ]:
# print (response)
print(response.choices[0].message.content)

## Parsing PDF where the text is not selectable and contains images

In [ ]:
import base64
import requests

# OpenAI API Key
api_key = os.getenv("OPENAI_KEY")


# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


# Path to your image
image_path = "pdf_image.jpg"

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {"Content-Type": "application/json", "Authorization": f"Bearer {api_key}"}

payload = {
    "model": "gpt-4o",
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": """You are an AI assistant tha read the content in the image. In the image you will have hand written text and you need to parse it carefully and return all the information in a JSON format and nothing else.
                    """,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                },
            ],
        }
    ],
    "max_tokens": 1000,
}


In [ ]:

response = requests.post(
    "https://api.openai.com/v1/chat/completions", headers=headers, json=payload
)


In [ ]:
# your structured data

response.json()["choices"][0]["message"]["content"]